# Import a model from a JSON file

In [1]:
from sigmaepsilon.solid import Structure
from sigmaepsilon.solid.fem import BernoulliFrame
import numpy as np
from numpy import pi as PI


In [2]:
from sigmaepsilon.solid import BeamSection, get_section

# all units in kN and cm

L = 150.  # length of the console [cm]
F = 1.  # value of the vertical load at the free end [kN]
E = 21000.0  # Young's modulus [kN/cm3]
nu = 0.3  # Poisson's ratio [-]
fy = 23.5  # yield strength [kN/cm2]
density = 7750.0 * 1e-6  # mass density [g/cm3]
weight = density * 1e-3 * 9.81  # [kN/cm3]
alpha = 1.2e-05
nElem = 20  # number of subdivisons to use

d_chs = 10.0  # diameter of the tube
t_chs = 1.0  # thickness of the tube
n_chs = 32  # number of subdivisions along the perimeter when discretizing the section
section = get_section('CHS', d=d_chs, t=t_chs, n=n_chs)
beam = BeamSection(wrap=section)
beam.calculate_geometric_properties()
props = beam.geometric_properties
A = props['A']  # area
Iy = props['Iy']  # second moment of inertia around the y axis
Iz = props['Iz']  # second moment of inertia around the z axis
Ix = props['Ix']  # torsional inertia


In [3]:
d_in = {
    "_version_": "0.0.1",

    "materials": {
        "Steel": {
            "E": E,
            "nu": nu,
            "fy": fy,
            "type": "steel",
            "density": density,
            "thermal-coefficient": alpha
        },
        "Custom": {
            "A": A,
            "Ix": Ix,
            "Iy": Iy,
            "Iz": Iz,
            "density": density,
            "thermal-coefficient": alpha
        }
    },

    "sections": {
        "Section": {
            "type": "CHS",
            "d": d_chs,
            "t": t_chs
        }
    },

    "frames": {
        "0": [
            [1.0, 0.0, 0.0],
            [0.0, 1.0, 0.0],
            [0.0, 0.0, 1.0]
        ]
    },

    "fixity": {
        "0": [True, True, True, True, True, True]
    },

    "loads": {
        "Fx": {
            "nodal": {
                "3": [F, 0.0, 0.0, 0.0, 0.0, 0.0]
            },
        },
        "Fy": {
            "nodal": {
                "3": [0.0, F, 0.0, 0.0, 0.0, 0.0]
            },
        },
        "Fz": {
            "nodal": {
                "3": [0.0, 0.0, F, 0.0, 0.0, 0.0]
            }
        }
    },

    "nodes": {
        "0": [0.0, 0.0, 0.0],
        "1": {
            "x": [L/3, 0.0, 0.0],
            "fixity": [False, False, False, False, False, False],
        },
        "2": {
            "x": [2*L/3, 0.0, 0.0],
        },
        "3": {
            "x": [L, 0.0, 0.0],
            "mass": 1.0
        }
    },

    "cells": {
        "0": {
            "nodes": [0, 1],
            "material": "Steel",
            "section": "Section",
            "density": density,
            "frame": [
                [1.0, 0.0, 0.0],
                [0.0, 1.0, 0.0],
                [0.0, 0.0, 1.0]
            ]
        },
        "1": {
            "nodes": [1, 2],
            "material": "Steel",
            "section": "Section",
            "density": density,
            "frame": "auto"
        },
        "2": {
            "nodes": [2, 3],
            "material": {
                "E": E,
                "nu": nu,
                "fy": fy,
                "type": "steel",
                "density": density,
                "thermal-coefficient": alpha
            },
            "section": {
                "type": "CHS",
                "d": d_chs,
                "t": t_chs
            },
            "density": "material",
            "frame": "auto"
        }
    },

    "data": {
        "nodes": {},
        "cells": {}
    },

}


In [4]:
d_out, mesh = BernoulliFrame.from_dict(d_in)
structure = Structure(mesh=mesh)
structure.linsolve()
dofsol = structure.nodal_dof_solution(store='dofsol')
print("Tip displacement : {}".format(dofsol[-1, 2, 2]))


Tip displacement : 0.18544389555161656


In [5]:
# Bernoulli solution
EI = E * Iy
sol_exact = F * L**3 / (3 * EI)
print("Tip displacement : {}".format(sol_exact))


Tip displacement : 0.1872413253058413


In [6]:
from sigmaepsilon.io import json2dict, dict2json

jsonpath = 'console_bernoulli.json'
dict2json(jsonpath, d_in)
d_in = json2dict(jsonpath)


In [7]:
d_out, mesh = BernoulliFrame.from_dict(d_in)
structure = Structure(mesh=mesh)
structure.linsolve()
dofsol = structure.nodal_dof_solution(store='dofsol')
print("Tip displacement : {}".format(dofsol[-1, 2, 2]))


Tip displacement : 0.18544389555161656
